In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException, JavascriptException, NoSuchElementException
import time
from datetime import date, datetime, timedelta 
import numpy as np
import pandas as pd
import os
import re
from functions import allMondays, intHolidayClosures, create_trips, country_holidays

In [3]:
#END DATE OF LAST REQUESTED TRIP
int_stop_date = date(2022, 12, 31)

In [4]:
def human():

    time.sleep(int(np.random.rand(1)*8))


In [47]:
#CONSTANTS
URL = 'https://camper.fti.de/'
DELAY = 3
PAGE_LOAD_ID = "SearchMask"
COOKIE_ID = '//*[@id="uc-center-container"]/div[2]/div/div[1]/div/button'
COOKIE_JS_PATH = 'document.querySelector("#usercentrics-root").shadowRoot.querySelector("#uc-center-container > div.sc-gWXbKe.fSIThp > div > div.sc-cxpSdN.ihpUDX > div > button")'
NOTIFICATION_JS_PATH = 'document.querySelector("#pid-14572 > div.cleverpush-confirm.cleverpush-confirm-modal.cleverpush-confirm-windows.cleverpush-confirm-chrome.cleverpush-confirm-chrome.cleverpush-confirm-1 > div.cleverpush-confirm-wrap > div.cleverpush-clearfix.cleverpush-clearfix-bottom > div.cleverpush-confirm-buttons > button.cleverpush-confirm-btn.cleverpush-confirm-btn-deny")'
POPUP_JS_PATH = 'document.querySelector("#trboModule_12920_202268_88_container > div.trbo-closeButton.trbo-theme-0.trboType-icon > i")'
CITY = 'Los Angeles'
CITY_JS_PATH = 'document.querySelector("#SearchMask > div.flexContainer.location > div:nth-child(1) > div:nth-child(1) > div > div > input")'
CAL_ELEMS_XPATH = "//span[contains(@class, 'cell day')]"
PU_CALENDAR_JS_PATH = 'document.querySelector("#SearchMask > div:nth-child(2) > div:nth-child(1) > div > div > div:nth-child(1) > input")'
DO_CALENDAR_JS_PATH = "document.querySelector('#SearchMask > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(1) > input')"
DO_MONTH_JS_PATH = "document.querySelector('#SearchMask > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(2) > header > span.day__month_btn.up')"
PU_MONTH_TEXT_CLASS_NAME = "vdp-datepicker__calendar"
PU_MONTH_NEXT_BTN_CLASS_NAME = "span[class = 'next']"
PU_MONTH_PREV_BTN_CLASS_NAME = "span[class = 'prev']"
DO_MONTH_PREV_BTN_CLASS_NAME = "span[class = 'prev']"
DO_MONTH_TEXT_XPATH = "//*[@id='SearchMask']/div[2]/div[2]/div/div/div[2]/header/span[3]"
SUBMIT_BUTTON_ID = "SubmitButton"
LOAD_MORE_RESULTS = "document.querySelector('#ResultList > button')"



_CITIES = [
            {'city':'San Francisco','state':'CA', 'abbr':'SFO', 'country':'USA'}#,
        #   {'city':'Las Vegas','state':'NV', 'abbr':'LAS', 'country':'USA'},
        #   {'city':'Los Angeles','state':'CA', 'abbr':'LAX', 'country':'USA'},
        #   {'city':'Seattle','state':'WA', 'abbr':'SEA', 'country':'USA'},
        #   {'city':'Salt Lake City','state':'UT', 'abbr':'SLC', 'country':'USA'},
        #   {'city':'Denver','state':'CO', 'abbr':'DEN', 'country':'USA'},
        #   {'city':'Chicago','state':'IL', 'abbr':'CHI', 'country':'USA'},
        #   {'city':'Dallas','state':'TX', 'abbr':'DFW', 'country':'USA'},
        #   {'city':'Orlando','state':'FL', 'abbr':'MCO', 'country':'USA'}
        ]


_GERMAN_MONTHS = {
    'Jan' : 'Jan',
    'Feb' : 'Feb',
    'Mar' : 'Mär',
    'Apr' : 'Apr',
    'May' : 'Mai',
    'Jun' : 'Jun',
    'Jul' : 'Jul',
    'Aug' : 'Aug',
    'Sep' : 'Sep',
    'Oct': 'Okt',
    'Nov': 'Nov',
    'Dec': 'Dez'

}


In [6]:
def create_browser(URL, ELEMENT_ID):
    
    #BOT DETECTION PREVENTION
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    #REMOVES POPUPS
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    options.add_experimental_option("prefs",prefs)

    #AVOIDS DEPTRECATION WARNING
    s=Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=s, options = options)

    browser.get(URL)

    try:
        myElem = WebDriverWait(browser, DELAY).until(EC.presence_of_element_located((By.ID, PAGE_LOAD_ID)))
        print ("Page is ready!")
    except TimeoutException:
        print ("Couldn't find element")

    return browser


browser = create_browser(URL, PAGE_LOAD_ID)

Page is ready!


In [ ]:
#CURRENTLY NOT REQUIRED
#deny news

browser.execute_script(f"return {NOTIFICATION_JS_PATH}").click()

#close popup

browser.execute_script(f"return {POPUP_JS_PATH}").click()

In [7]:
#COOKIES

browser.execute_script(f"return {COOKIE_JS_PATH}").click()

In [91]:
#reset calandar
def reset_calendar(cal_open_elem, cal_month_elem, cal_month_prev_button):
    browser.execute_script(f"return {cal_open_elem}").click()
    
    pickup = None

    if cal_open_elem[71] == '1':
        pickup = True

    elif cal_open_elem[71] == '2':
        pickup = False


    if pickup == True:

        start = 2
        end = 5
    else:
        start = 0
        end = 3

    
    if pickup == True:

        cal_current_year = browser.find_element(By.CLASS_NAME, f"{cal_month_elem}").text[6:10]

    else:

        cal_current_year = browser.execute_script(f" return {cal_month_elem}").text[4:10]
    
    
    currentYear = str(datetime.now().year)
    my_date = datetime.now()
    current_month = my_date.strftime("%B")[:3]

    

    if pickup == True:

        curr_cal_month = browser.find_element(By.CLASS_NAME, f"{cal_month_elem}").text[start:end]

    else:

        curr_cal_month = browser.execute_script(f" return {cal_month_elem}").text[start:end]



    german_current_month = _GERMAN_MONTHS[current_month]

    while german_current_month != curr_cal_month and cal_current_year != currentYear:

        print(f"Clicked {curr_cal_month} prev button")

        browser.find_element(By.CSS_SELECTOR, f"{cal_month_prev_button }").click()

        if pickup == True:

            curr_cal_month = browser.find_element(By.CLASS_NAME, f"{cal_month_elem}").text[start:end]

        else:

            curr_cal_month = browser.execute_script(f" return {cal_month_elem}").text[start:end]


        human()




    cal_elems = browser.find_elements(By.XPATH, f"{CAL_ELEMS_XPATH}")
    cal_elems_dict = {}
    for ele in cal_elems:

        if ele.text.isdigit():

            cal_elems_dict[ele.text] = ele
            
    cal_elems_dict[str(28)].click()




In [94]:
reset_calendar(PU_CALENDAR_JS_PATH, PU_MONTH_TEXT_CLASS_NAME, PU_MONTH_PREV_BTN_CLASS_NAME)

Clicked Apr prev button
Clicked Mär prev button
Clicked Feb prev button
Clicked Jan prev button
Clicked Dez prev button
Clicked Nov prev button


In [92]:
reset_calendar(DO_CALENDAR_JS_PATH, DO_MONTH_JS_PATH, DO_MONTH_PREV_BTN_CLASS_NAME)

Clicked Feb prev button
Clicked Jan prev button
Clicked Dez prev button
Clicked Nov prev button


In [83]:
human()

human()

browser.execute_script(f"return {COOKIE_JS_PATH}").click()

for city_index, city in enumerate(_CITIES):
    
    browser.execute_script(f"return {CITY_JS_PATH}").send_keys(f"{ city['city']}")

    human()

    browser.execute_script(f"return {CITY_JS_PATH}").send_keys(Keys.RETURN)

    human()


    US = True if city['country'] == 'USA' else False

    for trip_index, trip in enumerate(create_trips(int_stop_date, US)):

        print(f"Trip Mumber {trip_index}")
        #OPEN CALENDAR
        browser.execute_script(f"return {PU_CALENDAR_JS_PATH}").click()


        #print(f"SELECTING PICKUP MONTH FOR trip {trip_index}")

        human()

        #SELECT PICKUP MONTH
        pickup_month = browser.find_element(By.CLASS_NAME, f"{PU_MONTH_TEXT_CLASS_NAME}").text[2:5]


        trip['start_month'] = _GERMAN_MONTHS[trip['start_month']]
        trip['end_month'] = _GERMAN_MONTHS[trip['end_month']]

        while pickup_month != trip['start_month']:

            human()
        
            browser.find_element(By.CSS_SELECTOR, f"{PU_MONTH_NEXT_BTN_CLASS_NAME }").click()
            
            human()
            
            pickup_month = browser.find_element(By.CLASS_NAME, f"{PU_MONTH_TEXT_CLASS_NAME}").text[2:5]
            print(pickup_month)

        #print(f"SELECTING PICKUP DAY FOR trip {trip_index}")

        human()

        #click on the pickup day
        cal_elems = browser.find_elements(By.XPATH, f"{CAL_ELEMS_XPATH}")
        cal_elems_dict = {}
        for ele in cal_elems:

            if ele.text.isdigit():

                cal_elems_dict[ele.text] = ele
                
        cal_elems_dict[str(trip['start_day'])].click()


        print(f"SELECTING DROPOFF MONTH FOR trip {trip_index}")
        #open the drop off calendar

        browser.execute_script(f"return {DO_CALENDAR_JS_PATH}").click()

        human()

        #select the drop off month
        dropoff_month = browser.execute_script(f"return {DO_MONTH_JS_PATH}").text[:3]
        
        while dropoff_month != trip['end_month']:
            
            human()
            
            browser.find_element(By.XPATH, f"{DO_MONTH_TEXT_XPATH}").click()
            human()
            dropoff_month = browser.execute_script(f"return {DO_MONTH_JS_PATH}").text[:3]
            print(dropoff_month)

        #print(f"SELECTING DROPOFF DAY FOR TRIP {trip_index}")


        #select drop off day
        cal_elems = browser.find_elements(By.XPATH, f"{CAL_ELEMS_XPATH}")
        cal_elems_dict = {}
        for ele in cal_elems:

            if ele.text.isdigit():

                cal_elems_dict[ele.text] = ele
                
        cal_elems_dict[str(trip['end_day'])].click()

        #CLICK SEARCH AND LOAD PAGE


        #click search
        browser.find_element(By.ID, f"{SUBMIT_BUTTON_ID }").click()


        #SCROLL DOWN TO LOAD MORE RESULTS
        
        browser.execute_script("return window.scrollTo(0, document.body.scrollHeight);")

        human()

        browser.execute_script(f"return {LOAD_MORE_RESULTS}").click()


        #CHECK FOR PAGE LOAD, IF NO RESULTS...GO BACK?







        #SCRAPE
        LISTINGS_CLASS_NAME = "resultCard"
        listings = browser.find_elements(By.CLASS_NAME, f"{LISTINGS_CLASS_NAME}")


        trip_data = []
        for card in listings:

            vehicle_class = card.find_element(By.CLASS_NAME, "provider-data").text.split("\n")[0]
            company_name = card.find_element(By.CLASS_NAME, "providerLogo").get_attribute("alt")
            price_string = card.find_element(By.CLASS_NAME, "camper-details").find_element(By.CSS_SELECTOR, 'div.priceSummary.full > dl > dd').text.split('\n')
            total_price = '.'.join(re.findall(r'\d+', price_string[0]))
            daily_price = '.'.join(re.findall(r'\d+', price_string[1]))


            trip_data.append({
            'city': trip['city'],
            'start_date':trip['start_date'],
            'end_date':trip['end_date'],
            'total_price':total_price,
            'daily_price': daily_price,
            'vehicle_class':vehicle_class,
            'company_name':company_name
            })

        trip_data_df = pd.DataFrame(trip_data)



    
        #SAVE RESULTS TO FOLDER

        try: 
            os.makedirs(f"../Output Data/FTI Camper/{trip['city']} ")
        except FileExistsError:
            pass

        trip_data_df.to_csv(f"../Output Data/FTI Camper/{trip['city']}/{trip_index} ti {trip['city']} {date.today()}.csv", index=False)

    

    #SCROLL TO TOP
    browser.execute_script("return window.scrollTo(0,0)","")


    #CLEAR OLD CITY
    browser.execute_script(f"return {CITY_JS_PATH}").click()
    human()
    browser.execute_script(f"return {CITY_JS_PATH}").clear()


    #RESET CALENDARS

    reset_calendar(PU_CALENDAR_JS_PATH, PU_MONTH_TEXT_CLASS_NAME, PU_MONTH_PREV_BTN_CLASS_NAME)

    reset_calendar(DO_CALENDAR_JS_PATH, DO_MONTH_JS_PATH, DO_MONTH_PREV_BTN_CLASS_NAME)


    






    




Trip Mumber 0
SELECTING PICKUP MONTH FOR trip 0
SELECTING PICKUP DAY FOR trip 0
SELECTING DROPOFF MONTH FOR trip 0
SELECTING DROPOFF DAY FOR TRIP 0
Trip Mumber 1
SELECTING PICKUP MONTH FOR trip 1
Nov
SELECTING PICKUP DAY FOR trip 1
SELECTING DROPOFF MONTH FOR trip 1
SELECTING DROPOFF DAY FOR TRIP 1
Trip Mumber 2
SELECTING PICKUP MONTH FOR trip 2
SELECTING PICKUP DAY FOR trip 2
SELECTING DROPOFF MONTH FOR trip 2
Dez
SELECTING DROPOFF DAY FOR TRIP 2
Trip Mumber 3
SELECTING PICKUP MONTH FOR trip 3
SELECTING PICKUP DAY FOR trip 3
SELECTING DROPOFF MONTH FOR trip 3
SELECTING DROPOFF DAY FOR TRIP 3
Trip Mumber 4
SELECTING PICKUP MONTH FOR trip 4
SELECTING PICKUP DAY FOR trip 4
SELECTING DROPOFF MONTH FOR trip 4
SELECTING DROPOFF DAY FOR TRIP 4
Trip Mumber 5
SELECTING PICKUP MONTH FOR trip 5
Dez
SELECTING PICKUP DAY FOR trip 5
SELECTING DROPOFF MONTH FOR trip 5
SELECTING DROPOFF DAY FOR TRIP 5
Trip Mumber 6
SELECTING PICKUP MONTH FOR trip 6
SELECTING PICKUP DAY FOR trip 6
SELECTING DROPOFF MO

KeyboardInterrupt: 

### To-do

- reset calendar after each city
- wait for results to load, if results don't load or there are no results continue to next result
- test flaws
- combine csv files into one file for each city
- clean up into functions

In [6]:
#use if running code below more than once

browser.execute_script(f"return {CITY_JS_PATH}").click()
time.sleep(1)
browser.execute_script(f"return {CITY_JS_PATH}").clear()

In [15]:
#click search
browser.find_element(By.ID, f"{SUBMIT_BUTTON_ID }").click()

In [85]:
#ADD LOGIC TO WAIT FOR PAGE TO LOAD AND GO BACK IF THERE ARE NO RESULTS

#scroll page
browser.execute_script("return window.scrollTo(0, document.body.scrollHeight);")

In [17]:
#load all the results
browser.execute_script(f"return {LOAD_MORE_RESULTS}").click()

In [18]:
LISTINGS_CLASS_NAME = "resultCard"
listings = browser.find_elements(By.CLASS_NAME, f"{LISTINGS_CLASS_NAME}")
listings[0]

[<selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="17c0bfaf-03a6-4648-8b87-da2f789c695a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="67c4b55b-9c45-4825-8f87-8f368cd7b772")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="b74ec294-76da-4aff-8f44-74d1bfb2c929")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="2c50bde4-8830-4de6-9b64-a18ee8433cf6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="27d72101-b993-4cf3-b19a-aef121c54c1a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="4bfed701-d148-4c4d-a206-f644e5ef0f51")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="fc6f0ca4-a863-4ba8-b9d8-c6

In [19]:
trip_data = []
for card in listings:

    vehicle_class = card.find_element(By.CLASS_NAME, "provider-data").text.split("\n")[0]
    company_name = card.find_element(By.CLASS_NAME, "providerLogo").get_attribute("alt")
    price_string = card.find_element(By.CLASS_NAME, "camper-details").find_element(By.CSS_SELECTOR, 'div.priceSummary.full > dl > dd').text.split('\n')
    total_price = '.'.join(re.findall(r'\d+', price_string[0]))
    daily_price = '.'.join(re.findall(r'\d+', price_string[1]))


    trip_data.append({
    'city': trip['city'],
    'start_date':trip['start_date'],
    'end_date':trip['end_date'],
    'total_price':total_price,
    'daily_price': daily_price,
    'vehicle_class':vehicle_class,
    'company_name':company_name
    })

pd.DataFrame(trip_data)

,city,start_date,end_date,total_price,daily_price,vehicle_class,company_name
0,Los Angeles,2022-11-14,2022-12-01,816.00,74.18,Station Wagon,Travellers Autobarn
1,Los Angeles,2022-11-14,2022-12-01,1078.00,98.00,Motorhome C25,Road Bear
2,Los Angeles,2022-11-14,2022-12-01,1092.00,99.27,Motorhome M28,Mighty
3,Los Angeles,2022-11-14,2022-12-01,1111.00,101.00,Campervan B21,Road Bear
4,Los Angeles,2022-11-14,2022-12-01,1161.00,105.55,Hitop Campervan,Travellers Autobarn
5,Los Angeles,2022-11-14,2022-12-01,1161.00,105.55,Kuga Campervan,Travellers Autobarn
6,Los Angeles,2022-11-14,2022-12-01,1180.00,107.27,C Medium 24-26,Mighty
7,Los Angeles,2022-11-14,2022-12-01,1221.00,111.00,Motorhome C28,Road Bear
8,Los Angeles,2022-11-14,2022-12-01,1255.00,114.09,Motorhome Standard C25,Cruise
9,Los Angeles,2022-11-14,2022-12-01,1255.00,114.09,Motorhome Large C30,Cruise


In [20]:
trip_data_df = pd.DataFrame(trip_data)

In [21]:
try: 
    os.makedirs(f"../Output Data/FTI Camper/{trip['city']}")
except FileExistsError:
    pass

trip_data_df.to_csv(f"../Output Data/FTI Camper/{trip['city']}/{trip['city']} {date.today()}.csv", index=False)